API's Needed

In [ ]:
!pip install transformers sentencepiece datasets

Imports

In [ ]:
# Hugging face transformers to download pretrained model and tokenizer
import transformers
# Hugging face datasets to download the dataset
import datasets
# Pytorch for tensor
import torch
# For ploting the graph
import matplotlib.pyplot as plt
# Basic arithmatic operations
import numpy as np
# To show the progress bar
import tqdm
# For data handling
import itertools
# Specific functions from the libraries
from datasets import load_dataset
from transformers import AdamW, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import get_linear_schedule_with_warmup

# Model
Here we download the pre-trained model and tokenizer

In [ ]:
# model repository
model_repo = 'google/mt5-base'
# download mt5 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_repo)
# download model
model= AutoModelForSeq2SeqLM.from_pretrained(model_repo)
# puts model onto GPU
model = model.cuda()

# Dataset
Here we will be defining the dataset and downloading it

In [1]:
# Loading the dataset
dataset = load_dataset('alt')
# split the dataset into train validation and test
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

NameError: name 'load_dataset' is not defined

Adding the special tokens

In [ ]:
# add language token mapping to the tokenizer
LANG_TOKEN_MAPPING = {
    'en' : '<en>',
    'fil' : '<fil>',
    'hi' : '<hi>',
    'id' : '<id>',
    'ja' : '<ja>', 
}
# create a dict of the dict
special_tokens = { 'additional_special_tokens': list(LANG_TOKEN_MAPPING.values()) }
# add special tokens to the tokenizer
tokenizer.add_special_tokens(special_tokens)
# resize the token embeddings layer to correct size
model.resize_token_embeddings(len(tokenizer))

# Data Handling
Functions to handle the data

In [ ]:
# tokenizes and numericalizes input string
def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

# tokenizes and numericalizes target string
def encode_target_str(text, tokenizer, seq_len):
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)
  
  return token_ids[0]

# get all translations between all permutations of pairs of languages
def get_all_translation_data(translations, lang_token_map,
                            tokenizer, seq_length=20):
  input_ids = []
  target_ids = []
  
  langs = list(lang_token_map.keys())
  for input_lang, target_lang in itertools.permutations(langs, 2):
    input_text = translations[input_lang]
    target_text = translations[target_lang]
    
    if input_text is None or target_text is None:
        return None, None
    
    input_ids.append(encode_input_str(input_text, target_lang, tokenizer, seq_length, 
                                    lang_token_map))
    
    target_ids.append(encode_target_str(target_text, tokenizer, seq_length))
  
  return input_ids, target_ids

# generator function
def get_full_dataloader(dataset, lang_token_map, tokenizer, batch_size=32, num_workers=8):
    # get translations from the dataset
    dataset = train_dataset['translation']
    # intialize array
    data = []
    for example in dataset:
        # get translations for all permuations of languages
        input_id, target_id = get_all_translation_data(example, lang_token_map, tokenizer)
        # case where nothing is returned
        if input_id is None or target_id is None:
            continue
        # add the list of target and inputs 
        list_of_dicts = list(map(lambda x, y: {'input_ids': x, 'target_ids': y}, input_id, target_id))
        data = data + list_of_dicts
    # load dataset into a dataloader
    loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    # return the dataloader
    return loader

# Training

Evaluation function for validation

In [ ]:
def eval_model(model, gdataset, max_iters=8):
  test_generator = get_data_generator(gdataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)
  eval_losses = []
  for i, (input_batch, label_batch) in enumerate(test_generator):
    if i >= max_iters:
      break

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)
    eval_losses.append(model_out.loss.item())

  return np.mean(eval_losses)

In [ ]:
# Hyperparameters
EPOCHS = 5
batch_size = 64
learning_rate = 5e-3
n_batches = np.ceil(len(train_dataset) / batch_size)
total_steps = n_batches * EPOCHS
print_freq = total_steps / 100
checkpoint_freq = total_steps / 10
n_warmup_steps = int(0.01 * total_steps)

In [ ]:
# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
schedular = get_linear_schedule_with_warmup(optimizer, n_warmup_steps, total_steps)

Training the model

In [ ]:
loss_i = []

for epoch in range(EPOCHS):
    # Randomize data order, need to figure out a faster way to do this
    loader = get_full_dataloader(train_dataset, LANG_TOKEN_MAPPING, tokenizer, batch_size, num_workers=8)
    
    for i, batch in tqdm.tqdm(enumerate(loader), total = n_batches):
        inputs, targets = batch['input_ids'].cuda(), batch['target_ids'].cuda()
        
        # Zero gradients
        optimizer.zero_grad()
        # Forward pass (computes outputs and loss)
        output = model(input_ids=inputs, labels=targets)
        loss = output.loss
        # Back propagation (computes gradients)
        loss.backward()
        # Optimization and scheduling
        optimizer.step()
        # Adjust every 100 batches
        if(i+1) % 500 == 0:
            loss_i.append(loss.item())
            schedular.step()
        # prints training updates
        if (i+1) % print_freq == 0:
            print(f'Epoch: {epoch + 1}, Batch: {i+1}/{n_batches}, Loss: {loss.item()}, LR: {schedular.get_last_lr()[0]}')
        
        if (i + 1) % checkpoint_freq == 0:
            test_loss = eval_model(model, test_dataset)
            print('Saving model with test loss of {:.3f}'.format(test_loss))
            torch.save(model.state_dict(), 'mt5_translator.pt')
# Save the final model
torch.save(model.state_dict(), 'mt5_translator_final.pt')